In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('~mxnet/tools/')

In [2]:
import mxnet as mx

In [4]:
import os, urllib
def download(savepath,url):
    filename = url.split("/")[-1]
    if not os.path.exists(savepath+filename):
        urllib.urlretrieve(url, savepath+filename)
        
def get_model(model_path,prefix, epoch):
    download(model_path,prefix+'-symbol.json')
    download(model_path,prefix+'-%04d.params' % (epoch,))

In [5]:
model_path = '../models/'
get_model(model_path,'http://data.mxnet.io/models/imagenet-11k/resnet-152/resnet-152', 0)
get_model(model_path,'http://data.mxnet.io/models/imagenet/resnet/50-layers/resnet-50', 0)
get_model(model_path,'http://data.mxnet.io/models/imagenet/squeezenet/squeezenet_v1.0', 0)
get_model(model_path,'http://data.mxnet.io/models/imagenet/squeezenet/squeezenet_v1.1', 0)

In [100]:
full_synsets = [l.strip() for l in open(model_path+'resnet-152-synset.txt').readlines()]

full_synsets[8575].split(' ')[0]

'n09824609'

In [6]:
model_name = 'resnet-152'
sym, arg_params, aux_params = mx.model.load_checkpoint(model_path+model_name, 0)

In [ ]:
aux_params

In [73]:
mod = mx.mod.Module(symbol=sym, context=mx.cpu())

In [74]:
mod.bind(for_training=False, data_shapes=[('data', (batch_size,3,224,224))])

In [75]:
mod.set_params(arg_params, aux_params)

In [ ]:
mx.viz.plot_network(sym)

In [76]:
data_shape=(3, 224, 224)
num_classes = 178
batch_per_gpu = 8
num_gpus = 1


batch_size = batch_per_gpu * num_gpus

In [77]:
val = mx.io.ImageRecordIter(
    path_imgrec         = '../data/raw/acorn squash_n07717410/acorn_val.rec',
    data_name           = 'data',
    label_name          = 'softmax_label',
    batch_size          = batch_size,
    data_shape          = data_shape,
    rand_crop           = False,
    rand_mirror         = False
        )

In [80]:
%%time

a = mod.predict(val)

CPU times: user 47.7 s, sys: 13.4 s, total: 1min 1s
Wall time: 15.6 s


In [79]:
np.argmax(a.asnumpy(), axis=1)

array([ 636, 6343, 7506, 9989, 7516, 9989, 7516, 7516, 9989, 9989, 7515,
       7516, 9989, 7516, 7516, 9989, 7516, 7709, 7516, 7516, 7509, 9989,
       9989, 9989, 9989, 7516, 7515, 7516, 9989, 7516, 7516, 9989, 9994,
       7509, 7516, 7516, 7516, 7516, 9989, 9989, 9989, 7517, 9989, 7516,
       7516, 9989, 7516, 7516, 7515, 9989, 7516, 7516, 7516, 7516])

In [81]:
pred = a.asnumpy()
ipred.argsort()[-5:][::-1]

array([7516, 9989, 7515, 9988, 7517])

In [85]:
cats= []
for ipred in pred:
    cats.append( 7516 in ipred.argsort()[-5:][::-1])
round(float(sum(cats))/len(cats),2)

0.87